In [ ]:
!pip install -q --upgrade langchain langchain-openai langchain-core langchain_community docx2txt pypdf langchain_chroma sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.2/440.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.5/305.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 38.0 MB/s eta

In [ ]:
import langchain

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "personal"

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")
llm_response = llm.invoke("Tell me a joke")
llm_response

AIMessage(content='Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 11, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BoDdnOHW20MBGip8pJ5zozBazQBwM', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='run--8dde2e67-8f09-4730-869f-a1b6b349ea48-0', usage_metadata={'input_tokens': 11, 'output_tokens': 17, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser = StrOutputParser()
output_parser.invoke(llm_response)

'Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!'

In [ ]:
from langchain_community.document_loaders import Docx2txtLoader,PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from typing import List
from langchain_core.documents import Document

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
)

In [ ]:
def load_documents(folder_path:str) -> List[Document]:
  documents=[]
  for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path,filename)
    if filename.endswith('.pdf'):
      loader = PyPDFLoader(file_path)
    elif filename.endswith('.docx'):
      loader = Docx2txtLoader(file_path)
    else:
      continue
    documents.extend(loader.load())
  return documents

folder_path = '/content/sample_data'
documents = load_documents(folder_path)

print(f'Loaded {len(documents)} documents from the folder')
splits = text_splitter.split_documents(documents)
print(f'Split into {len(splits)} chunks')

Loaded 71 documents from the folder
Split into 237 chunks


In [ ]:
embdeddings = OpenAIEmbeddings()
document_embeddings = embdeddings.embed_documents([split.page_content for split in splits])


In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


/tmp/ipython-input-11-2976604365.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or dat

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain_chroma import Chroma
embedding_function = OpenAIEmbeddings()
collection_name = "my_collection"
vectorstore = Chroma.from_documents(collection_name=collection_name,documents=splits,embedding=embedding_function,)

ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [ ]:
query = " "
search_results = vectorstore.similarity_search(query,k=2)
print(f"Top 2 relevant are \n")
for i ,result in enumerate(search_results,1):
  print(f"Result {i}:")
  print(f"Source: {result.metadata.get('source','Unknown')}")
  print(f'Content: {result.page_content}')


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


Top 2 relevant are 

Result 1:
Source: /content/sample_data/orwellanimalfarm.pdf
Content: until today, no animal had killed another animal. Not even a rat had
been killed. They had made their way on to the little knoll where the
half-finished windmill stood, and with one accord they all lay down as
though huddling together for warmth-Clover, Muriel, Benjamin, the
cows, the sheep, and a whole flock of geese and hens-everyone, indeed,
except the cat, who had suddenly disappeared just before Napoleon
ordered the animals to assemble. For some time nobody spoke. Only
Boxer remained on his feet. He fidgeted to and fro, swishing his long
black tail against his sides and occasionally uttering a little whinny of
surprise. Finally he said:
"I do not understand it. I would not have believed that such things could
happen on our farm. It must be due to some fault in ourselves. The
solution, as I see it, is to work harder. From now onwards I shall get up
a full hour earlier in the mornings."
And he 

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs = {'k':2})
retriever.invoke("How were winters in animal farm")

[Document(id='264a78a4-7165-4d09-8ef6-6a4bacb8789d', metadata={'page': 53, 'title': 'Animal Farm by George Orwell', 'creationdate': 'D:20010220101733', 'page_label': '54', 'moddate': '2001-02-20T10:18:08-05:00', 'creator': 'PyPDF', 'source': '/content/sample_data/orwellanimalfarm.pdf', 'total_pages': 71, 'producer': 'Acrobat Web Capture 4.0'}, page_content="disappear indoors again. But in the morning a deep silence hung over\nthe farmhouse. Not a pig appeared to be stirring. It was nearly nine\no'clock when Squealer made his appearance, walking slowly and\ndejectedly, his eyes dull, his tail hanging limply behind him, and with\nevery appearance of being seriously ill. He called the animals together\nand told them that he had a terrible piece of news to impart. Comrade\nNapoleon was dying!\nA cry of lamentation went up. Straw was laid down outside the doors of\nthe farmhouse, and the animals walked on tiptoe. With tears in their\neyes they asked one another what they should do if their 

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
template = """
Answer the question based only on the following context:
{context}
Question : {question}

Answer:
"""
prompt = ChatPromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
rag_chain = (
    {"context": retriever , "question":RunnablePassthrough()} | prompt
)


In [ ]:
def docs2string(docs):
  return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
rag_chain = ({"context":retriever | docs2string,"question":RunnablePassthrough()} | prompt )
rag_chain.invoke("What was the constitution of Animal Farm")


ChatPromptValue(messages=[HumanMessage(content='\nAnswer the question based only on the following context:\ntheir own benefit. So that, what with the songs, the processions,\nSquealer\'s lists of figures, the thunder of the gun, the crowing of the\ncockerel, and the fluttering of the flag, they were able to forget that\ntheir bellies were empty, at least part of the time.\nIn April, Animal Farm was proclaimed a Republic, and it became\nnecessary to elect a President. There was only one candidate, Napoleon,\nwho was elected unanimously. On the same day it was given out that\nfresh documents had been discovered which revealed further details\nabout Snowball\'s complicity with Jones. It now appeared that Snowball\nhad not, as the animals had previously imagined, merely attempted to\nlose the Battle of the Cowshed by means of a stratagem, but had been\nopenly fighting on Jones\'s side. In fact, it was he who had actually been\nthe leader of the human forces, and had charged into battle wit

In [ ]:
rag_chain = (
             {"context":retriever
             |docs2string,"question":RunnablePassthrough()}
             |prompt
             |llm
             |StrOutputParser()
             )
question = "What was the constitution of Animal Farm"
response = rag_chain.invoke(question)
print(response)

The constitution of Animal Farm was based on the principles outlined in the Seven Commandments, which were established following the animals' rebellion against human rule. These commandments encapsulated the ideals of Animalism, promoting equality and the notion that all animals are equal. Over time, however, the interpretation and implementation of these commandments became distorted under the leadership of Napoleon, particularly as he consolidated power and manipulated the animals' perceptions to maintain control.
